# Dataset pre-processing

## Dataset format convertion

```
Timers for logger timer :
- Dataset convertion : 58min 48sec
  - convert_format executed 1202 times : 58min 47sec (2.934 sec / exec)
    - CT convertion executed 1202 times : 27min 27sec (1.371 sec / exec)
      - Sanity check executed 5 times : 264 ms (52 ms / exec)
      - Original data loading executed 1197 times : 13min 41sec (686 ms / exec)
      - Data convertion + saving executed 1197 times : 10min 35sec (530 ms / exec)
    - Mask convertion executed 1202 times : 31min 19sec (1.564 sec / exec)
      - Sanity check executed 7 times : 860 ms (122 ms / exec)
      - Original data loading executed 1195 times : 12min 46sec (641 ms / exec)
      - Data convertion + saving executed 1195 times : 13min 36sec (683 ms / exec)
```

In [12]:
path = '/storage/Totalsegmentator_dataset/s0001/masks.npz'

with np.load(path) as file:
    print(file.files)
    print(file['indices'].shape)

['indices', 'pixdim', 'shape']
(1925885, 4)


In [1]:
import os
import glob
import logging
import importlib
import numpy as np
import tensorflow as tf

from tqdm import tqdm

from loggers import set_level, timer
from datasets import get_dataset
from utils.med_utils import data_io

try:
    tf.config.set_visible_devices([], 'GPU')
except:
    pass

importlib.reload(data_io)

time_logger = logging.getLogger('timer')

def _maybe_convert(filename, out_file, overwrite = False, check = False, debug = False):
    out_file = filename.replace(os.path.basename(filename), out_file)
    
    formatted_out_file = out_file
    if out_file.endswith(('.tensor', '.stensor', '.npy')):
        if debug: print('Output tensor requires a specific format, formatting {}'.format(filename))
        formatted_out_file = data_io.format_filename(
            out_file,
            shape      = data_io.get_shape(filename),
            voxel_dims = data_io.get_voxel_dims(filename)
        )
    
    if debug: print('Formatted output file : {}'.format(formatted_out_file))
    
    if os.path.exists(formatted_out_file):
        if not overwrite:
            if debug: print('Output file already exists !')
            if not check: return True
            try:
                time_logger.start_timer('Sanity check')
                data_io.load_medical_data(formatted_out_file)
                if debug: print('Successfully loaded data in the existing file !\n')
                return True
            except Exception as e:
                print('[ERROR] While loading file {}\n  --> {}'.format(formatted_out_file, e))
            finally:
                time_logger.stop_timer('Sanity check')
        os.remove(formatted_out_file)
    
    time_logger.start_timer('Original data loading')
    data, voxel_dims = data_io.load_medical_data(filename)
    time_logger.stop_timer('Original data loading')
    
    time_logger.start_timer('Data convertion + saving')
    res = data_io.save_medical_data(filename = out_file, data = data, voxel_dims = voxel_dims)
    time_logger.stop_timer('Data convertion + saving')

    if debug: print('Result output file : {}\n'.format(res))

    return True

@timer
def convert_format(row, data_format = None, seg_format = None, ** kwargs):
    data_file = row['images']
    seg_file  = row['segmentation']
    
    success = True
    if data_format:
        time_logger.start_timer('CT convertion')
        success = success and _maybe_convert(data_file, data_format, ** kwargs)
        time_logger.stop_timer('CT convertion')
    
    if seg_format:
        time_logger.start_timer('Mask convertion')
        success = success and _maybe_convert(seg_file, seg_format, ** kwargs)
        time_logger.stop_timer('Mask convertion')
    
    return success


dataset = get_dataset('total_segmentator')
dataset = dataset[dataset['segmentation'].apply(lambda f: f.endswith('.npz'))]

set_level('time')

time_logger.start_timer('Dataset convertion')

labels = list(sorted(dataset.loc[0, 'label']))
for i, (_, row) in tqdm(enumerate(dataset.iloc[:].iterrows()), total = len(dataset)):
    print('Patient {} / {}'.format(i + 1, len(dataset)))
    
    convert_format(row, data_format = 'ct.tensor', seg_format = 'masks.stensor', overwrite = False, check = True, debug = True)

time_logger.stop_timer('Dataset convertion')
time_logger.log_time()


2023-06-13 09:30:03.232451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 09:30:03.330649: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-13 09:30:03.354862: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. E

Loading dataset total_segmentator...


  0%|                                                                                          | 0/1202 [00:00<?, ?it/s]

Patient 1 / 1202
Output tensor requires a specific format, formatting /storage/Totalsegmentator_dataset/s0001/ct.nii.gz
Formatted output file : /storage/Totalsegmentator_dataset/s0001/ct-1.5-1.5-1.5-249-188-213.tensor
Output file already exists !
Successfully loaded data in the existing file !

Output tensor requires a specific format, formatting /storage/Totalsegmentator_dataset/s0001/masks.npz
Formatted output file : /storage/Totalsegmentator_dataset/s0001/masks-1.5-1.5-1.5-249-188-213-104.stensor
Output file already exists !


2023-06-13 09:30:17.660491: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
  0%|▎                                                                                 | 4/1202 [00:00<03:13,  6.18it/s]

...

Result output file : /storage/Totalsegmentator_dataset/s1338/masks-1.5-1.5-1.5-135-135-249-104.stensor

Patient 1201 / 1202
Output tensor requires a specific format, formatting /storage/Totalsegmentator_dataset/s1400/ct.nii.gz
Formatted output file : /storage/Totalsegmentator_dataset/s1400/ct-1.5-1.5-1.5-271-271-459.tensor
Result output file : /storage/Totalsegmentator_dataset/s1400/ct-1.5-1.5-1.5-271-271-459.tensor

Output tensor requires a specific format, formatting /storage/Totalsegmentator_dataset/s1400/masks.npz
Formatted output file : /storage/Totalsegmentator_dataset/s1400/masks-1.5-1.5-1.5-271-271-459-104.stensor


100%|██████████████████████████████████████████████████████████████████████████████▉| 1201/1202 [58:43<00:03,  3.65s/it]

Result output file : /storage/Totalsegmentator_dataset/s1400/masks-1.5-1.5-1.5-271-271-459-104.stensor

Patient 1202 / 1202
Output tensor requires a specific format, formatting /storage/Totalsegmentator_dataset/s1403/ct.nii.gz
Formatted output file : /storage/Totalsegmentator_dataset/s1403/ct-1.5-1.5-1.5-258-258-317.tensor
Result output file : /storage/Totalsegmentator_dataset/s1403/ct-1.5-1.5-1.5-258-258-317.tensor

Output tensor requires a specific format, formatting /storage/Totalsegmentator_dataset/s1403/masks.npz
Formatted output file : /storage/Totalsegmentator_dataset/s1403/masks-1.5-1.5-1.5-258-258-317-104.stensor


100%|███████████████████████████████████████████████████████████████████████████████| 1202/1202 [58:48<00:00,  2.94s/it]

Result output file : /storage/Totalsegmentator_dataset/s1403/masks-1.5-1.5-1.5-258-258-317-104.stensor

Timers for logger timer :
- Dataset convertion : 58min 48sec
  - convert_format executed 1202 times : 58min 47sec (2.934 sec / exec)
    - CT convertion executed 1202 times : 27min 27sec (1.371 sec / exec)
      - Sanity check executed 5 times : 264 ms (52 ms / exec)
      - Original data loading executed 1197 times : 13min 41sec (686 ms / exec)
      - Data convertion + saving executed 1197 times : 10min 35sec (530 ms / exec)
    - Mask convertion executed 1202 times : 31min 19sec (1.564 sec / exec)
      - Sanity check executed 7 times : 860 ms (122 ms / exec)
      - Original data loading executed 1195 times : 12min 46sec (641 ms / exec)
      - Data convertion + saving executed 1195 times : 13min 36sec (683 ms / exec)


In [3]:
from datasets import get_dataset

get_dataset('total_segmentator')['segmentation'].apply(lambda f: f.split('.')[-1]).value_counts()

Loading dataset total_segmentator...


stensor    1202
gz            1
Name: segmentation, dtype: int64

## TotalSegmentator dataset normalization (mask combination)

In [ ]:
import os
import glob
import importlib
import numpy as np
import nibabel as nib
import tensorflow as tf

from tqdm import tqdm

from datasets import get_dataset
from utils.med_utils import load_medical_data, data_io

try:
    tf.config.set_visible_devices([], 'GPU')
except:
    pass

importlib.reload(data_io)

def combine_masks(masks, labels, out_file, overwrite = False, check = False):
    if os.path.exists(out_file):
        if not overwrite:
            if not check: return True
            try:
                if 'nii' in out_file:
                    file = nib.load(out_file)
                    file.get_fdata('unchanged')
                elif out_file.endswith('npz'):
                    with np.load(out_file) as file:
                       file['mask']
                return True
            except Exception as e:
                print('[ERROR] Error when loading file {} : {}'.format(out_file, e))
                os.remove(out_file)
    elif out_file.endswith('tensor'):
        if glob.glob(out_file.replace('masks', 'masks*')): return True
        
    ext = out_file.split('.')[-1]
    if os.path.exists(out_file.replace(ext, 'nii.gz')):
        data = nib.load(out_file.replace(ext, 'nii.gz'))
        
        comb_mask, affine, headers = data.get_fdata(), data.affine, data.header
    else:
        comb_mask, affine, headers, segmented = None, None, None, []
        for i, label in enumerate(tqdm(labels)):
            if label not in masks:
                print('[WARNING] Label {} is not in masks !'.format(label))
                continue

            file = nib.load(masks[label])
            mask = file.get_fdata()
            if comb_mask is None:
                comb_mask = np.zeros(mask.shape + (len(labels), ), dtype = np.uint8)
                affine, headers = file.affine, file.header

            if mask.shape != comb_mask.shape[:-1]:
                raise ValueError('Mask of shape {} != reference mask {}'.format(mask.shape, comb_mask.shape))

            mask_bool = mask > 0.5
            indexes   = np.where(mask)
            if np.any(comb_mask[indexes + (i, )] != 0):
                raise ValueError('Overlap with label : {} and {}'.format(label, labels[np.max(comb_mask[indexes + (i, )])]))

            if np.any(mask_bool):
                segmented.append(label)

            comb_mask[indexes + (i, )] = 1
            assert np.all(comb_mask[..., i] == mask)

        if comb_mask is None:
            raise RuntimeError('No valid mask !')

    result = data_io.save_medical_data(
        out_file,
        comb_mask,
        voxel_dims = headers['pixdim'][1:4],
        affine = affine,
        header = headers,
        labels = labels
    )
    return True

dataset = get_dataset('total_segmentator')
dataset = dataset[dataset['segmentation'].apply(lambda f: f.endswith('.npz'))]

labels = list(sorted(dataset.loc[0, 'label']))
for i, (_, row) in tqdm(enumerate(dataset.iloc[:].iterrows()), total = len(dataset)):
    print('Patient {} / {}'.format(i + 1, len(dataset)))
    
    seg_files = row['segmentation'] if isinstance(row['segmentation'], list) else []
    out_file  = row['images'].replace('ct.nii.gz', 'masks.stensor')
    
    res = combine_masks(
        {organ : file for organ, file in zip(row['label'], row['segmentation'])},
        labels, out_file = out_file, overwrite = False, check = False
    )


Loading dataset total_segmentator...


  0%|                                                                                          | 0/1202 [00:00<?, ?it/s]

Patient 1 / 1202
Patient 2 / 1202
Patient 3 / 1202
Patient 4 / 1202
Patient 5 / 1202
Patient 6 / 1202
Patient 7 / 1202
Patient 8 / 1202
/storage/Totalsegmentator_dataset/s0011/masks.stensor


In [13]:
import importlib
import tensorflow as tf

from utils import is_equal
from utils.med_utils import data_io

importlib.reload(data_io)

file1 = '/storage/Totalsegmentator_dataset/s0001/masks.npz'
file2 = '/storage/Totalsegmentator_dataset/s0001/masks-1.5-1.5-1.5-249-188-213-104.stensor'

d1, _ = data_io.load_medical_data(file1, use_sparse = True, dtype = tf.uint8)
d2, _ = data_io.load_medical_data(file2, use_sparse = True, dtype = tf.uint8)

print(is_equal(d1.indices, d2.indices)[1])

Value are equals !


In [3]:
%%time

import numpy as np
import nibabel as nib
import tensorflow as tf

path_npz = '/storage/Totalsegmentator_dataset/s0110/masks.npz'
path_npz = dataset.iloc[7]['segmentation'].replace('.nii.gz', '.npz')
path_nii = path_npz.replace('.npz', '.nii.gz')

with np.load(path_npz) as data_npz:
    indexes = data_npz['mask']
    mask_npz = tf.sparse.SparseTensor(
        indices = indexes, values = tf.ones((len(indexes), ), dtype = tf.uint8), dense_shape = data_npz['shape']
    )


CPU times: user 958 ms, sys: 909 ms, total: 1.87 s
Wall time: 2.82 s


2023-04-05 19:29:35.173483: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 19:29:36.726308: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14783 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-04-05 19:29:36.727064: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14485 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:65:00.0, compute capability: 7.5


In [4]:
%%time

data_nii = nib.load(path_nii)
mask_nii = data_nii.get_fdata(caching = 'unchanged').astype(np.uint8)

CPU times: user 10.1 s, sys: 3.42 s, total: 13.5 s
Wall time: 13.6 s


## Dataset extraction

In [2]:
import os
import zipfile
import nibabel as nib

from tqdm import tqdm

path = '/storage'
zip_filename = '../Totalsegmentator_dataset.zip'

with zipfile.ZipFile(zip_filename, 'r') as file:
    for name in tqdm(file.namelist()[:]):
        filename = os.path.join(path, name)
        if name.endswith('/'):
            os.makedirs(filename[:-1], exist_ok = True)
            continue
        elif os.path.exists(filename):
            try:
                if 'ct' in filename:
                    nib.load(filename).get_fdata(caching = 'unchanged')
                continue
            except Exception as e:
                print('[ERROR] Error while loading {} : {}'.format(filename, e))
                os.remove(filename)
        
        try:
            file.extract(name, path = path)
        except Exception as e:
            print('[ERROR] when extracting {} : {}'.format(name, e))


 60%|█████████████████████████████████████████████▏                             | 77685/128830 [04:43<03:20, 254.85it/s]

[ERROR] Error while loading /storage/Totalsegmentator_dataset/s0864/ct.nii.gz : Compressed file ended before the end-of-stream marker was reached


 60%|█████████████████████████████████████████████▎                             | 77899/128830 [04:45<04:10, 203.15it/s]

[ERROR] when extracting Totalsegmentator_dataset/s0864/ct.nii.gz : Bad CRC-32 for file 'Totalsegmentator_dataset/s0864/ct.nii.gz'


100%|██████████████████████████████████████████████████████████████████████████| 128830/128830 [08:14<00:00, 260.78it/s]
